
# First Steps

## Simulation Overview
The first call of **MERA** will compile the package!

In [1]:
using Mera

┌ Info: Precompiling Mera [02f895e8-fdb1-4346-8fe6-c721699f5126]
└ @ Base loading.jl:1273



*__   __ _______ ______   _______ 
|  |_|  |       |    _ | |   _   |
|       |    ___|   | || |  |_|  |
|       |   |___|   |_||_|       |
|       |    ___|    __  |       |
| ||_|| |   |___|   |  | |   _   |
|_|   |_|_______|___|  |_|__| |__|



Get information with the function ``getinfo`` about the simulation for a selected output and assign it to an object, here: "info"  (composite type). The RAMSES output folders are assumed to be in the current working directory, and the user can give a relative or absolute path. The information is read from several files: info-file, header-file, from the header of the Fortran binary files of the first CPU (hydro, grav, part, clump, sink, ... if they exist), etc. Many familiar names and acronyms known from RAMSES are maintained. The function ``getinfo`` prints a small summary and the given units are printed in human-readable representation.

In [2]:
info = getinfo(420, "../../testing/simulations/manu_sim_sf_L10"); # output=400 in given path

[Mera]: 2020-02-15T22:02:55.164

Code: RAMSES
output [420] summary:
mtime: 2017-07-27T01:22:09
ctime: 2019-12-24T09:57:04.822
simulation time: 624.91 [Myr]
boxlen: 48.0 [kpc]
ncpu: 1024
ndim: 3
-------------------------------------------------------
amr:           true
level(s): 6 - 10 --> cellsize(s): 750.0 [pc] - 46.88 [pc]
-------------------------------------------------------
hydro:         true
hydro-variables:  6  --> (:rho, :vx, :vy, :vz, :p, :var6)
hydro-descriptor: (:density, :velocity_x, :velocity_y, :velocity_z, :thermal_pressure, :passive_scalar_1)
γ: 1.01
-------------------------------------------------------
gravity:       true
gravity-variables: (:epot, :ax, :ay, :az)
-------------------------------------------------------
particles:     true
particle variables: (:vx, :vy, :vz, :mass, :birth)
-------------------------------------------------------
clumps:        true
clump-variables: (:index, :lev, :parent, :ncell, :peak_x, :peak_y, :peak_z, Symbol("rho-"), Symbol("rho

The simulation output can be selected in several ways, which is realised by using multiple dispatch. See the different defined methods on the function ``getinfo``:

In [3]:
# info = getinfo(); # default: output=1 in current folder, 
# info = getinfo("../simulations/"); # given path, default: output=1
# info = getinfo(output=400, path="../simulations/"); # pass path and output number by keywords
methods(getinfo)

# 4 methods for generic function "getinfo":
[1] getinfo(; output, path, namelist, verbose) in Mera at /Users/mabe/Documents/Projects/dev/Mera/src/read_data/RAMSES/getinfo.jl:51
[2] getinfo(path::String; output, namelist, verbose) in Mera at /Users/mabe/Documents/Projects/dev/Mera/src/read_data/RAMSES/getinfo.jl:46
[3] getinfo(output::Real; path, namelist, verbose) in Mera at /Users/mabe/Documents/Projects/dev/Mera/src/read_data/RAMSES/getinfo.jl:38
[4] getinfo(output::Real, path::String; namelist, verbose) in Mera at /Users/mabe/Documents/Projects/dev/Mera/src/read_data/RAMSES/getinfo.jl:42

## Fields
The created object ``info`` is of type ``InfoType`` (composite type):

In [4]:
typeof(info)

InfoType

The previously printed information and even more simulation properties are assigned to the object and can be accessed from fields and sub-fields.
Get an overview with:

In [5]:
viewfields(info);

output	= 420
path	= /Users/mabe/Documents/Projects/dev/Mera/tutorials/version_1/../../testing/simulations/manu_sim_sf_L10
fnames ==> subfields: (:output, :info, :amr, :hydro, :hydro_descriptor, :gravity, :particles, :part_descriptor, :clumps, :timer, :header, :namelist, :compilation, :makefile, :patchfile)

simcode	= RAMSES
mtime	= 2017-07-27T01:22:09
ctime	= 2019-12-24T09:57:04.822
ncpu	= 1024
ndim	= 3
levelmin	= 6
levelmax	= 10
boxlen	= 48.0
time	= 41.9092891721775
aexp	= 1.0
H0	= 1.0
omega_m	= 1.0
omega_l	= 0.0
omega_k	= 0.0
omega_b	= 0.0
unit_l	= 3.085677581282e21
unit_d	= 6.76838218451376e-23
unit_m	= 1.9885499720830952e42
unit_v	= 6.557528732282063e6
unit_t	= 4.70554946422349e14
gamma	= 1.01
hydro	= true
nvarh	= 6
nvarp	= 5
variable_list	= Symbol[:rho, :vx, :vy, :vz, :p, :var6]
gravity_variable_list	= Symbol[:epot, :ax, :ay, :az]
particles_variable_list	= Symbol[:vx, :vy, :vz, :mass, :birth]
clumps_variable_list	= Symbol[:index, :lev, :parent, :ncell, :peak_x, :peak_y, :peak_z, S

Get a simple list of the fields of any object:

In [6]:
propertynames(info)

(:output, :path, :fnames, :simcode, :mtime, :ctime, :ncpu, :ndim, :levelmin, :levelmax, :boxlen, :time, :aexp, :H0, :omega_m, :omega_l, :omega_k, :omega_b, :unit_l, :unit_d, :unit_m, :unit_v, :unit_t, :gamma, :hydro, :nvarh, :nvarp, :variable_list, :gravity_variable_list, :particles_variable_list, :clumps_variable_list, :sinks_variable_list, :descriptor, :amr, :gravity, :particles, :clumps, :sinks, :rt, :namelist, :namelist_content, :headerfile, :makefile, :files_content, :timerfile, :compilationfile, :patchfile, :Narraysize, :scale, :grid_info, :part_info, :compilation, :constants)

## Physical Units
All calculations in **MERA** are processed in the code units of the loades simulation. The **RAMSES** scaling factors from code- to cgs-units are given for the length, density, mass, velocity and time, assigned to the fields: unit_l, unit_d, unit_m, unit_v, unit_t

To make life easier, we provide more predefined scaling factors, assigned to the sub-field ``scale``:

In [7]:
viewfields(info.scale) 


[Mera]: Fields to scale from user/code units to selected units
Mpc	= 0.0010000000000006482
kpc	= 1.0000000000006481
pc	= 1000.0000000006482
mpc	= 1.0000000000006482e6
ly	= 3261.5637769461323
Au	= 2.0626480623310105e23
km	= 3.0856775812820004e16
m	= 3.085677581282e19
cm	= 3.085677581282e21
mm	= 3.085677581282e22
μm	= 3.085677581282e25
Mpc3	= 1.0000000000019446e-9
kpc3	= 1.0000000000019444
pc3	= 1.0000000000019448e9
mpc3	= 1.0000000000019446e18
ly3	= 3.469585750743794e10
Au3	= 8.775571306099254e69
km3	= 2.9379989454983075e49
m3	= 2.9379989454983063e58
cm3	= 2.9379989454983065e64
mm3	= 2.937998945498306e67
μm3	= 2.937998945498306e76
Msol_pc3	= 0.9997234790001649
g_cm3	= 6.76838218451376e-23
Msol_pc2	= 999.7234790008131
g_cm2	= 0.20885045168302602
Gyr	= 0.014910986463557083
Myr	= 14.910986463557084
yr	= 1.4910986463557083e7
s	= 4.70554946422349e14
ms	= 4.70554946422349e17
Msol	= 9.99723479002109e8
Mearth	= 3.329677459032007e14
Mjupiter	= 1.0476363431814971e12
g	= 1.9885499720830952e42
km_

In [8]:
list_field = propertynames( info.scale )

(:Mpc, :kpc, :pc, :mpc, :ly, :Au, :km, :m, :cm, :mm, :μm, :Mpc3, :kpc3, :pc3, :mpc3, :ly3, :Au3, :km3, :m3, :cm3, :mm3, :μm3, :Msol_pc3, :g_cm3, :Msol_pc2, :g_cm2, :Gyr, :Myr, :yr, :s, :ms, :Msol, :Mearth, :Mjupiter, :g, :km_s, :m_s, :cm_s, :nH, :erg, :g_cms2, :T_mu, :Ba)

The underline in the unit representation corresponds to the fraction line, e.g.:
 
|field name | corresponding unit |
|---- | ----|
|Msol_pc3        | Msol * pc^-3|
|g_cm3          | g * cm^-3 |
|Msol_pc2        | Msol * pc^-2|
|g_cm2           | g * cm^-2|
|            km_s| km * s^-1|
|             m_s| m * s^-1|
|            cm_s| cm * s^-1|
|          g_cms2| g / (cm * s^2)|
|          nH    | cm^-3 |
|          T_mu  | T / μ |
|          Ba    | = Barye (pressure) [cm^-1 * g * s^-2] |

Access a scaling factor to use it in your calculations or plots by e.g.:

In [9]:
info.scale.km_s  

65.57528732282063

To reduce the hierarchy of sub-fields, assign a new object:

In [10]:
scale = info.scale;

The scaling factor can now be accessed by:

In [11]:
scale.km_s

65.57528732282063

Furthermore, the scales can be assigned by applying the function ``createscales`` on an object of type ``InfoType`` (here: `info`):

In [12]:
typeof(info)

InfoType

In [13]:
my_scales = createscales(info)
my_scales.km_s

65.57528732282063

## Physical Constants
Some useful constants are assigned to the `InfoType` object:

In [14]:
viewfields(info.constants)


[Mera]: Constants given in cgs units
Au	= 0.01495978707
Mpc	= 3.08567758128e24
kpc	= 3.08567758128e21
pc	= 3.08567758128e18
mpc	= 3.08567758128e15
ly	= 9.4607304725808e17
Msol	= 1.9891e33
Mearth	= 5.9722e27
Mjupiter	= 1.89813e30
Rsol	= 6.96e10
me	= 9.1093897e-28
mp	= 1.6726231e-24
mn	= 1.6749286e-24
mH	= 1.66e-24
amu	= 1.6605402e-24
NA	= 6.0221367e23
c	= 2.99792458e10
G	= 6.67259e-8
kB	= 1.38062e-16
Gyr	= 3.15576e16
Myr	= 3.15576e13
yr	= 3.15576e7



Reduce the hierarchy of sub-fields:

In [15]:
con = info.constants;

In [16]:
viewfields(con)


[Mera]: Constants given in cgs units
Au	= 0.01495978707
Mpc	= 3.08567758128e24
kpc	= 3.08567758128e21
pc	= 3.08567758128e18
mpc	= 3.08567758128e15
ly	= 9.4607304725808e17
Msol	= 1.9891e33
Mearth	= 5.9722e27
Mjupiter	= 1.89813e30
Rsol	= 6.96e10
me	= 9.1093897e-28
mp	= 1.6726231e-24
mn	= 1.6749286e-24
mH	= 1.66e-24
amu	= 1.6605402e-24
NA	= 6.0221367e23
c	= 2.99792458e10
G	= 6.67259e-8
kB	= 1.38062e-16
Gyr	= 3.15576e16
Myr	= 3.15576e13
yr	= 3.15576e7



## InfoType Fields Overview
All fields and sub-fields that are assigned to the `InfoType` or from other objects can be viewed by the function **viewfields**, **namelist**, **makefile**, **timerfile**, **patchfile**.
See the methods list:

In [17]:
methods(viewfields)

# 10 methods for generic function "viewfields":
[1] viewfields(object::PhysicalUnitsType) in Mera at /Users/mabe/Documents/Projects/dev/Mera/src/functions/viewfields.jl:181
[2] viewfields(object::Mera.FilesContentType) in Mera at /Users/mabe/Documents/Projects/dev/Mera/src/functions/viewfields.jl:166
[3] viewfields(object::DescriptorType) in Mera at /Users/mabe/Documents/Projects/dev/Mera/src/functions/viewfields.jl:150
[4] viewfields(object::FileNamesType) in Mera at /Users/mabe/Documents/Projects/dev/Mera/src/functions/viewfields.jl:134
[5] viewfields(object::CompilationInfoType) in Mera at /Users/mabe/Documents/Projects/dev/Mera/src/functions/viewfields.jl:116
[6] viewfields(object::GridInfoType) in Mera at /Users/mabe/Documents/Projects/dev/Mera/src/functions/viewfields.jl:90
[7] viewfields(object::PartInfoType) in Mera at /Users/mabe/Documents/Projects/dev/Mera/src/functions/viewfields.jl:73
[8] viewfields(object::ScalesType) in Mera at /Users/mabe/Documents/Projects/dev/Mera/src/functions/viewfields.jl:57
[9] viewfields(object::InfoType) in Mera at /Users/mabe/Documents/Projects/dev/Mera/src/functions/viewfields.jl:12
[10] viewfields(object::DataSetType) in Mera at /Users/mabe/Documents/Projects/dev/Mera/src/functions/viewfields.jl:197

In [18]:
methods(namelist)

# 2 methods for generic function "namelist":
[1] namelist(object::Dict{Any,Any}) in Mera at /Users/mabe/Documents/Projects/dev/Mera/src/functions/viewfields.jl:244
[2] namelist(object::InfoType) in Mera at /Users/mabe/Documents/Projects/dev/Mera/src/functions/viewfields.jl:226

Get a detailed overview of all the fields from MERA composite types:

In [19]:
viewallfields(info)

output	= 420
path	= /Users/mabe/Documents/Projects/dev/Mera/tutorials/version_1/../../testing/simulations/manu_sim_sf_L10
fnames ==> subfields: (:output, :info, :amr, :hydro, :hydro_descriptor, :gravity, :particles, :part_descriptor, :clumps, :timer, :header, :namelist, :compilation, :makefile, :patchfile)

simcode	= RAMSES
mtime	= 2017-07-27T01:22:09
ctime	= 2019-12-24T09:57:04.822
ncpu	= 1024
ndim	= 3
levelmin	= 6
levelmax	= 10
boxlen	= 48.0
time	= 41.9092891721775
aexp	= 1.0
H0	= 1.0
omega_m	= 1.0
omega_l	= 0.0
omega_k	= 0.0
omega_b	= 0.0
unit_l	= 3.085677581282e21
unit_d	= 6.76838218451376e-23
unit_m	= 1.9885499720830952e42
unit_v	= 6.557528732282063e6
unit_t	= 4.70554946422349e14
gamma	= 1.01
hydro	= true
nvarh	= 6
nvarp	= 5
variable_list	= Symbol[:rho, :vx, :vy, :vz, :p, :var6]
gravity_variable_list	= Symbol[:epot, :ax, :ay, :az]
particles_variable_list	= Symbol[:vx, :vy, :vz, :mass, :birth]
clumps_variable_list	= Symbol[:index, :lev, :parent, :ncell, :peak_x, :peak_y, :peak_z, S

## Disc Space
Gives an overview of the used disc space for the different data types of the selected output:

In [20]:
storageoverview(info)

Overview of the used disc space for output: [420]
------------------------------------------------------
Folder:         1.38 GB 	<282.44 KB>/file
AMR-Files:      321.44 MB 	<321.44 KB>/file
Hydro-Files:    607.0 MB 	<606.41 KB>/file
Gravity-Files:  485.14 MB 	<485.14 KB>/file
Particle-Files: 188.0 KB 	<188.0 Bytes>/file
Clump-Files:    184.25 KB 	<184.25 Bytes>/file


mtime: 2017-07-27T01:22:09
ctime: 2019-12-24T09:57:04.822
